# fetch_risk_free_rate.ipynb

In [66]:
def fetch_risk_free_rate(date_str=None):
    """
    Fetch the risk-free rate using the given date or default to the latest rate.

    Parameters:
    -----------
    date_str : str - Date in format '%Y-%m-%d'. Defaults to None.

    Returns:
    --------
    float - Risk-free rate.
    """
    treasury_rate = yf.Ticker("^TNX").history(start='1900-01-01')
    treasury_rate.index = pd.to_datetime(treasury_rate.index).tz_localize(None)
    if date_str:
        date = datetime.strptime(date_str, "%Y-%m-%d")
        nearest_index = treasury_rate.index.get_indexer([pd.Timestamp(date)], method='nearest')[0]
        return treasury_rate.iloc[nearest_index]['Close'] / 100
    return treasury_rate['Close'].iloc[-1] / 100

# Black_Scholes_Option_Pricing_Model.ipynb

In [67]:
#Imports

import math
from scipy.stats import norm
import yfinance as yf
import pandas as pd
from datetime import datetime

In [68]:


def option_price(S: float, K: float, T: float, v: float, r: float, option_type: str) -> float:
    """
    Calculate the price of an option using the Black-Scholes formula.

    Parameters:
    S : float - Current stock price
    K : float - Strike price
    T : float - Time to maturity in years
    v : float - Volatility of the underlying asset
    r : float - Risk-free interest rate
    option_type : str - 'C' for Call or 'P' for Put

    Returns:
    float - Option price
    """
    # Basic checks
    if T <= 0:
        raise ValueError("Time to maturity must be positive.")
    if v <= 0:
        raise ValueError("Volatility must be positive.")
    
    # Calculate d1: Normalized distance between S and K, adjusted for volatility and time.
    d1 = (math.log(S / K) + (r + 0.5 * v ** 2) * T) / (v * math.sqrt(T))

    # Calculate d2: Offset of d1 by the volatility term, accounting for the time to maturity.
    d2 = d1 - (v * math.sqrt(T))

    if option_type.upper() == 'C':
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type.upper() == 'P':
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Use 'C' for Call or 'P' for Put.")
  
    return option_price


In [69]:

# Fetch the risk-free rate
r = fetch_risk_free_rate(date_str="2024-01-01")

# Calculate Call Price
call_price = option_price(S=480, K=410, T=0.008, v=0.9443, r=r, option_type='C')
print(f"Call Option Price: {call_price:.2f}")

# Calculate Put Price
put_price = option_price(S=100, K=100, T=1, v=0.2, r=r, option_type='P')
print(f"Put Option Price: {put_price:.2f}")

Call Option Price: 70.58
Put Option Price: 6.03


# Calculate_Implied_Volatility_Market.ipynb

In [70]:
from math import sqrt, log, exp
from scipy.stats import norm
from scipy.optimize import newton
from datetime import datetime
import pandas as pd

In [71]:
def calculate_implied_volatility(S: float, K: float, T: float, market_price: float, r: float, option_type: str, method: str = "newton") -> float:
    """
    Calculate the implied volatility using Black-Scholes formula and Newton-Raphson method or Bisection Method.

    Parameters:
    -----------
    S : float - Current stock price.
    K : float - Strike price.
    T : float - Time to maturity in years.
    market_price : float - Market price of the option.
    option_type : str - 'C' for Call or 'P' for Put.
    r : float - Risk-free interest rate.
    method: str - Method of calculation (newton or bisection)

    Returns:
    --------
    float
        Implied volatility or a failure message if the calculation does not converge.
    """
    
    # Define the objective function for Newton-Raphson
    def objective_function(volatility):
        """
        Calculates the difference between Black-Scholes price and market price.

        Parameters:
        -----------
        volatility : float - Implied volatility guess.

        Returns:
        --------
        float - Price difference.
        """
        price = option_price(S, K, T, volatility, r, option_type,)
        return price - market_price

    # Define vega (sensitivity to volatility)
    def vega(volatility):
        """
        Vega of the option, the sensitivity of option price to changes in volatility.

        Parameters:
        -----------
        volatility : float - Implied volatility guess.

        Returns:
        --------
        float
            Vega value.
        """
        d1 = (log(S / K) + (r + 0.5 * volatility ** 2) * T) / (volatility * sqrt(T))
        return S * sqrt(T) * norm.pdf(d1)

    # Initial guess for Newton-Raphson
    volatility_guess = 0.2

    if method == "newton":
        try:
            implied_vol = newton(objective_function, volatility_guess, fprime=vega, maxiter=100)
            if implied_vol < 0:
                raise ValueError("Negative implied vol found, invalid solution.")
            return implied_vol
        
        except RuntimeError:

            # Alternatively implement bisection if needed
            low, high = 0.0001, 5.0
            for _ in range(200):
                mid = 0.5*(low+high)
                f_mid = objective_function(mid)
                if abs(f_mid) < 1e-6:
                    return mid
                f_low = objective_function(low)
                if f_mid * f_low > 0:
                    low = mid
                else:
                    high = mid
            raise ValueError("Bisection method and Newton's method did not converge.")
    
        

In [72]:
# Fetch the risk-free rate
r = fetch_risk_free_rate(date_str="2025-01-10")

# Calculate implied volatility
implied_volatility = calculate_implied_volatility(S=480, K=410, T=0.008, market_price=70.58, r=r, option_type='C')

# Display the implied volatility
print(f"Implied Volatility: {implied_volatility:.4f}")

Implied Volatility: 0.9358


# fetch_option_chain.ipynb

In [73]:
import yfinance as yf
from datetime import date
from typing import Optional, List, Dict

In [74]:


def fetch_option_chain(ticker: str, 
                       expiry_date: Optional[date] = None) -> List[Dict]:
    """
    Description:
        Fetch an option chain for a given ticker (and optional expiration date).

    Parameters:
        ticker : str - The stock ticker symbol (e.g., 'AAPL').
        expiry_date : Optional[date] - The expiration date for the options in YYYY-MM-DD format.
            If None, fetches all available options.

    Returns:
        List[Dict]: A list of dictionaries where each dictionary represents an option contract
        (calls and puts) with keys like strike price, bid, ask, volume, openInterest, and type ('C' for call, 'P' for put).

    """
    try:
        # Instantiate yfinance object
        ticker_obj = yf.Ticker(ticker)
        
        # Validate ticker and get available expiration dates
        if not ticker_obj.options:
            raise ValueError(f"No option chains available for ticker '{ticker}'.")
        
        all_expirations = ticker_obj.options

        # Function to filter relevant columns
        def filter_columns(dataframe, option_type, expiry):
            dataframe = dataframe[["strike", "lastPrice", "bid", "ask", "volume", "openInterest", "impliedVolatility", "inTheMoney"]].copy()
            dataframe.loc[:, "type"] = option_type
            dataframe.loc[:, "expiry"] = expiry
            return dataframe.to_dict(orient='records')

        # If expiry_date is None, fetch all available expirations
        if not expiry_date:
            all_options = []
            for exp_str in all_expirations:
                try:
                    calls = filter_columns(ticker_obj.option_chain(exp_str).calls, "C", exp_str)
                    puts = filter_columns(ticker_obj.option_chain(exp_str).puts, "P", exp_str)
                    all_options.extend(calls)
                    all_options.extend(puts)
                except Exception as e:
                    print(f"Error fetching options for expiration {exp_str}: {e}")
            return all_options
        
        # If expiry_date is specified, validate it
        exp_str = expiry_date.strftime('%Y-%m-%d')
        if exp_str not in all_expirations:
            raise ValueError(f"Expiration date '{exp_str}' is not available. "
                             f"Available expirations: {all_expirations}")
        
        # Fetch calls and puts for the specified expiration date
        calls = filter_columns(ticker_obj.option_chain(exp_str).calls, "C", exp_str)
        puts = filter_columns(ticker_obj.option_chain(exp_str).puts, "P", exp_str)

        result = []
        result.extend(calls)
        result.extend(puts)
        return result

    except ValueError as ve:
        print(f"ValueError: {ve}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

In [75]:
def test_fetch_option_chain():
    """
    Test the fetch_option_chain function and display results for valid scenarios.
    """
    ticker = "AAPL"  # Example stock ticker

    # Test: Fetch all option chains for a ticker
    try:
        print("Fetching all options for the ticker...")
        all_options = fetch_option_chain(ticker)
        if all_options:
            print(f"Example data (Total fetched: {len(all_options)}):")
            print(all_options[:5])  # Display first 5 option contracts
        else:
            print("No options found.")
    except Exception as e:
        print(f"Test failed: {e}")

    # Test: Fetch options for a specific expiration date
    try:
        print("\nFetching options for a specific expiration date...")
        ticker_obj = yf.Ticker(ticker)
        available_expirations = ticker_obj.options
        if available_expirations:
            expiry_date = date.fromisoformat(available_expirations[0])  # Use the first available expiration
            specific_options = fetch_option_chain(ticker, expiry_date)
            if specific_options:
                print(f"Example data for {expiry_date} (Total fetched: {len(specific_options)}):")
                print(specific_options[:5])  # Display first 5 option contracts
            else:
                print("No options found for the specified expiration.")
        else:
            print("No expiration dates available.")
    except Exception as e:
        print(f"Test failed: {e}")

# Run the tests
if __name__ == "__main__":
    test_fetch_option_chain()

Fetching all options for the ticker...
Example data (Total fetched: 2024):
[{'strike': 5.0, 'lastPrice': 229.75, 'bid': 227.75, 'ask': 228.6, 'volume': 1, 'openInterest': 1567, 'impliedVolatility': 20.50000359375, 'inTheMoney': True, 'type': 'C', 'expiry': '2025-01-17'}, {'strike': 10.0, 'lastPrice': 240.87, 'bid': 222.4, 'ask': 223.65, 'volume': 1, 'openInterest': 29, 'impliedVolatility': 16.51562983886719, 'inTheMoney': True, 'type': 'C', 'expiry': '2025-01-17'}, {'strike': 15.0, 'lastPrice': 218.6, 'bid': 217.35, 'ask': 218.65, 'volume': 8, 'openInterest': 168, 'impliedVolatility': 14.125001171875, 'inTheMoney': True, 'type': 'C', 'expiry': '2025-01-17'}, {'strike': 20.0, 'lastPrice': 216.57, 'bid': 212.4, 'ask': 213.65, 'volume': 25, 'openInterest': 69, 'impliedVolatility': 12.531252167968752, 'inTheMoney': True, 'type': 'C', 'expiry': '2025-01-17'}, {'strike': 25.0, 'lastPrice': 216.26, 'bid': 207.3, 'ask': 208.65, 'volume': 1, 'openInterest': 77, 'impliedVolatility': 11.328127919

# Binomial_Tree_Options_Pricing_Model.ipynb

In [76]:

import numpy as np
import math

In [77]:
def binomial_tree(S: float, K: float, T: float, r: float, N: int, v: float, option_type: str) -> float:
    """
    Calculate the option price using the Binomial Tree method, with parameters aligned to Black-Scholes.
    
    Parameters:
    S: float - Current stock price
    K: float - Strike price
    T: float - Time to maturity in years
    r: float - Risk-free interest rate
    N: int - Number of steps in the binomial tree
    v: float - Volatility of the underlying asset 
    option_type: str - 'C' for Call or 'P' for Put 
    
    Returns:
    float - Option price
    """

    if N <= 0:
            raise ValueError("Number of steps must be > 0.")

    # Calculate time step and risk-neutral probabilities

    dt = T / N                              # Length of each time step
    u = math.exp(v * math.sqrt(dt))         # Upward movement factor
    d = 1 / u                               # Downward movement factor (ensures no arbitrage)
    p = (math.exp(r * dt) - d) / (u - d)    # Risk-neutral probability
    q = 1 - p                               # Complement probability
    
    # Initialize asset prices at maturity
    asset_prices = S * d**(np.arange(N, -1, -1)) * u**(np.arange(0, N+1, 1))
    
    # Initialize option values at maturity
    if option_type.upper() == 'C':  # Call option
        option_values = np.maximum(asset_prices - K, 0)
    elif option_type.upper() == 'P':  # Put option
        option_values = np.maximum(K - asset_prices, 0)
    else:
        raise ValueError("Invalid option_type. Use 'C' for Call or 'P' for Put.")
    
    # Perform backward induction
    for i in range(N, 0, -1):
        option_values = math.exp(-r * dt) * (p * option_values[1:i+1] + q * option_values[0:i])
    
    return option_values[0]

In [78]:
# Fetch the risk-free rate
r = fetch_risk_free_rate(date_str="2024-01-01")

# Call Option Example
call_price = binomial_tree(S=100, K=100, T=1, r=r, N=3, v=0.2, option_type='C')
print(f"Call Option Price: {call_price:.2f}")

# Put Option Example
put_price = binomial_tree(S=100, K=100, T=1, r=r, N=3, v=0.2, option_type='P')
print(f"Put Option Price: {put_price:.2f}")

Call Option Price: 10.52
Put Option Price: 6.65


# Calculate_Volatility_Historical.ipynb

In [79]:
import yfinance as yf
import numpy as np
import datetime
from datetime import date

In [80]:
def calculate_annualized_volatility(ticker: str, start_date: date, end_date: date) -> float:
    
    stock_data = yf.download(ticker, start=start_date, end=end_date)

    stock_data['log_returns'] = np.log(stock_data['Close'] / stock_data['Close'].shift(1)) 
    
    daily_volatility = stock_data['log_returns'].std()

    annualized_volatility = daily_volatility * np.sqrt(252)

    return annualized_volatility



In [81]:
ticker = 'TSLA'
start_date = '2024-04-01'
end_date = '2024-08-01'


volatility = calculate_annualized_volatility(ticker, start_date, end_date)
print(f"Annualized Volatility for {ticker} is {volatility:.2%}")

Annualized Volatility for TSLA is 64.18%


[*********************100%***********************]  1 of 1 completed


# get_similar_historical_options.ipynb

In [3]:
from typing import Tuple, List, Dict, Optional
from datetime import date
import requests

In [4]:
# Fetch data from Polygon.io or use sample data for historical options
def get_similar_historical_options(ticker: str,
                                   expiry_days: int,
                                   strike_ratio_range: Tuple[float, float],
                                   volatility_range: Tuple[float, float],
                                   date_range: Tuple[date, date],
                                   extra_filters: Optional[Dict] = None,
                                   api_key: str = "cg9uTxvW3V9CpMAawm7v8ohoCercAI6Y") -> List[Dict]:
    """
    Identify historically similar options based on specified criteria.
    Use sample data for demonstration purposes when API calls are unavailable.
    """
    try:
        # Placeholder for using sample data instead of API calls
        if True:  # Replace with `if not use_api` when ready to toggle
            print("Using sample data...")
            results = []

            # Filter sample data for the specified ticker
            ticker_data = next((data for data in sample_data if data["ticker"] == ticker), None)
            if not ticker_data:
                print(f"No sample data found for ticker {ticker}.")
                return []

            for option in ticker_data["options"]:
                try:
                    
                    # Get Stock Ticker
                    option["ticker"] = ticker

                    # Get expiration date
                    option_expiry_date = date.fromisoformat(option["details"]["expiration_date"])

                    # Ensure expiration date is within the provided historical date range
                    if not (date_range[0] <= option_expiry_date <= date_range[1]):
                        continue

                    # Calculate strike ratio
                    underlying_price = option.get("day", {}).get("close", 1)  # Use 'close' price
                    strike_price = option["details"]["strike_price"]
                    strike_ratio = strike_price / underlying_price
                    if not (strike_ratio_range[0] <= strike_ratio <= strike_ratio_range[1]):
                        continue

                    # Check implied volatility
                    implied_volatility = option.get("implied_volatility", 0)
                    if not (volatility_range[0] <= implied_volatility <= volatility_range[1]):
                        continue

                    # Apply additional filters if provided
                    if extra_filters:
                        match = all(option.get(key) == value for key, value in extra_filters.items())
                        if not match:
                            continue

                    results.append(option)
                except Exception as e:
                    print(f"Error filtering sample data: {e}")

            print(f"Total matching options from sample data: {len(results)}")
            return results

        # Uncomment and use for API calls when ready
        """
        base_quote_url = "https://api.polygon.io/v3/quotes"
        base_contract_url = "https://api.polygon.io/v3/snapshot/options"
        results = []

        # Define start and end dates in ISO format
        start_date = date_range[0].isoformat()
        end_date = date_range[1].isoformat()

        # Step 1: Fetch option tickers (Quotes API)
        quote_params = {
            "ticker": f"O:{ticker}",
            "timestamp.gte": start_date,
            "timestamp.lte": end_date,
            "limit": 1000,
            "apiKey": api_key,
        }

        print(f"Making API request to Quotes endpoint: {base_quote_url}")
        quote_response = requests.get(base_quote_url, params=quote_params)
        quote_response.raise_for_status()
        quote_data = quote_response.json()

        if not quote_data.get("results"):
            print(f"No quote data found for ticker {ticker} in the given date range.")
            return []

        print(f"Fetched {len(quote_data['results'])} quotes for {ticker}.")

        # Step 2: Fetch detailed data for each option contract (Option Contract API)
        for option in quote_data["results"]:
            option_ticker = option.get("optionsTicker")
            if not option_ticker:
                continue

            contract_url = f"{base_contract_url}/{ticker}/{option_ticker}"
            contract_params = {"apiKey": api_key}

            print(f"Fetching detailed contract data for {option_ticker}")
            contract_response = requests.get(contract_url, params=contract_params)
            contract_response.raise_for_status()
            contract_data = contract_response.json()

            if not contract_data.get("results"):
                continue

            contract_details = contract_data["results"]

            # Step 3: Filter results based on criteria
            try:
                expiration_date = date.fromisoformat(contract_details["details"]["expiration_date"])
                days_to_expiry = (expiration_date - date.today()).days
                if not (expiry_days - 5 <= days_to_expiry <= expiry_days + 5):
                    continue

                underlying_price = contract_details.get("underlying_asset", {}).get("price", 1)  # Avoid division by zero
                strike_price = contract_details["details"]["strike_price"]
                strike_ratio = strike_price / underlying_price
                if not (strike_ratio_range[0] <= strike_ratio <= strike_ratio_range[1]):
                    continue

                implied_volatility = contract_details.get("implied_volatility", 0)
                if not (volatility_range[0] <= implied_volatility <= volatility_range[1]):
                    continue

                if extra_filters:
                    for key, value in extra_filters.items():
                        if contract_details.get(key) != value:
                            break

                results.append(contract_details)

            except KeyError as ke:
                print(f"Missing key in contract data: {ke}")
            except Exception as e:
                print(f"Error processing contract data: {e}")

        print(f"Total filtered results: {len(results)}")
        return results
        """

    except Exception as e:
        print(f"An error occurred while processing options data: {e}")
        return []

In [5]:
sample_data = [
    # Stock 1: Apple Inc. (AAPL) - Technology Sector
    {
        "ticker": "AAPL",
        "options": [
            {
                "option_id": "O:AAPL240119C00150000",
                "break_even_price": 155.50,
                "day": {
                    "change": 2.30,
                    "change_percent": 1.5,
                    "close": 153.20,
                    "high": 155.60,
                    "low": 151.50,
                    "open": 152.00,
                    "volume": 1400,
                    "vwap": 153.00,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-01-19",
                    "shares_per_contract": 100,
                    "strike_price": 150.00,
                },
                "greeks": {
                    "delta": 0.68,
                    "gamma": 0.045,
                    "theta": -0.02,
                    "vega": 0.14,
                    "rho": 0.04,
                },
                "implied_volatility": 0.24,
                "last_quote": {
                    "ask": 5.10,
                    "bid": 4.90,
                    "bid_size": 20,
                    "ask_size": 15,
                },
                "open_interest": 3400,
            },
            {
                "option_id": "O:AAPL240616P00140000",
                "break_even_price": 138.00,
                "day": {
                    "change": -1.80,
                    "change_percent": -1.2,
                    "close": 139.20,
                    "high": 141.00,
                    "low": 138.00,
                    "open": 140.50,
                    "volume": 1100,
                    "vwap": 139.60,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2024-06-16",
                    "shares_per_contract": 100,
                    "strike_price": 140.00,
                },
                "greeks": {
                    "delta": -0.50,
                    "gamma": 0.040,
                    "theta": -0.016,
                    "vega": 0.11,
                    "rho": -0.03,
                },
                "implied_volatility": 0.20,
                "last_quote": {
                    "ask": 3.80,
                    "bid": 3.50,
                    "bid_size": 18,
                    "ask_size": 12,
                },
                "open_interest": 2900,
            },
            {
                "option_id": "O:AAPL241220C00160000",
                "break_even_price": 164.50,
                "day": {
                    "change": 2.80,
                    "change_percent": 1.75,
                    "close": 162.20,
                    "high": 165.00,
                    "low": 161.00,
                    "open": 161.50,
                    "volume": 1700,
                    "vwap": 162.80,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-12-20",
                    "shares_per_contract": 100,
                    "strike_price": 160.00,
                },
                "greeks": {
                    "delta": 0.75,
                    "gamma": 0.043,
                    "theta": -0.018,
                    "vega": 0.12,
                    "rho": 0.045,
                },
                "implied_volatility": 0.26,
                "last_quote": {
                    "ask": 6.30,
                    "bid": 6.00,
                    "bid_size": 22,
                    "ask_size": 18,
                },
                "open_interest": 3700,
            },
            {
                "option_id": "O:AAPL231117P00130000",
                "break_even_price": 128.50,
                "day": {
                    "change": -2.00,
                    "change_percent": -1.5,
                    "close": 129.50,
                    "high": 131.00,
                    "low": 128.00,
                    "open": 130.50,
                    "volume": 800,
                    "vwap": 129.90,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2023-11-17",
                    "shares_per_contract": 100,
                    "strike_price": 130.00,
                },
                "greeks": {
                    "delta": -0.55,
                    "gamma": 0.050,
                    "theta": -0.017,
                    "vega": 0.10,
                    "rho": -0.025,
                },
                "implied_volatility": 0.22,
                "last_quote": {
                    "ask": 4.20,
                    "bid": 4.00,
                    "bid_size": 15,
                    "ask_size": 10,
                },
                "open_interest": 2500,
            },
            {
                "option_id": "O:AAPL230421C00170000",
                "break_even_price": 172.50,
                "day": {
                    "change": 1.90,
                    "change_percent": 1.1,
                    "close": 171.00,
                    "high": 173.50,
                    "low": 170.50,
                    "open": 171.80,
                    "volume": 1200,
                    "vwap": 171.60,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2023-04-21",
                    "shares_per_contract": 100,
                    "strike_price": 170.00,
                },
                "greeks": {
                    "delta": 0.79,
                    "gamma": 0.046,
                    "theta": -0.025,
                    "vega": 0.14,
                    "rho": 0.06,
                },
                "implied_volatility": 0.28,
                "last_quote": {
                    "ask": 7.50,
                    "bid": 7.20,
                    "bid_size": 12,
                    "ask_size": 8,
                },
                "open_interest": 3100,
            },
        ],
    },
    # Remaining stocks: XOM (Energy), JNJ (Healthcare), HD (Consumer Discretionary), and JPM (Financials)
    # Each stock should follow the same pattern, with diverse expirations and highly realistic stats.

        {
        "ticker": "XOM",
        "options": [
            {
                "option_id": "O:XOM240119C00100000",
                "break_even_price": 105.50,
                "day": {
                    "change": 1.80,
                    "change_percent": 1.75,
                    "close": 104.20,
                    "high": 106.00,
                    "low": 103.50,
                    "open": 103.80,
                    "volume": 2000,
                    "vwap": 104.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-01-19",
                    "shares_per_contract": 100,
                    "strike_price": 100.00,
                },
                "greeks": {
                    "delta": 0.72,
                    "gamma": 0.048,
                    "theta": -0.023,
                    "vega": 0.15,
                    "rho": 0.05,
                },
                "implied_volatility": 0.27,
                "last_quote": {
                    "ask": 5.80,
                    "bid": 5.50,
                    "bid_size": 25,
                    "ask_size": 20,
                },
                "open_interest": 4000,
            },
            {
                "option_id": "O:XOM240616P00090000",
                "break_even_price": 88.00,
                "day": {
                    "change": -2.00,
                    "change_percent": -2.22,
                    "close": 89.20,
                    "high": 90.50,
                    "low": 88.00,
                    "open": 89.50,
                    "volume": 1500,
                    "vwap": 89.10,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2024-06-16",
                    "shares_per_contract": 100,
                    "strike_price": 90.00,
                },
                "greeks": {
                    "delta": -0.52,
                    "gamma": 0.035,
                    "theta": -0.020,
                    "vega": 0.12,
                    "rho": -0.04,
                },
                "implied_volatility": 0.24,
                "last_quote": {
                    "ask": 4.10,
                    "bid": 3.80,
                    "bid_size": 18,
                    "ask_size": 15,
                },
                "open_interest": 3100,
            },
            {
                "option_id": "O:XOM241220C00120000",
                "break_even_price": 125.80,
                "day": {
                    "change": 3.20,
                    "change_percent": 2.65,
                    "close": 124.50,
                    "high": 126.50,
                    "low": 123.50,
                    "open": 124.00,
                    "volume": 1800,
                    "vwap": 124.80,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-12-20",
                    "shares_per_contract": 100,
                    "strike_price": 120.00,
                },
                "greeks": {
                    "delta": 0.76,
                    "gamma": 0.046,
                    "theta": -0.022,
                    "vega": 0.16,
                    "rho": 0.055,
                },
                "implied_volatility": 0.29,
                "last_quote": {
                    "ask": 7.80,
                    "bid": 7.50,
                    "bid_size": 20,
                    "ask_size": 18,
                },
                "open_interest": 4500,
            },
            {
                "option_id": "O:XOM231117P00110000",
                "break_even_price": 108.00,
                "day": {
                    "change": -1.50,
                    "change_percent": -1.35,
                    "close": 109.00,
                    "high": 110.50,
                    "low": 107.50,
                    "open": 109.50,
                    "volume": 1200,
                    "vwap": 109.30,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2023-11-17",
                    "shares_per_contract": 100,
                    "strike_price": 110.00,
                },
                "greeks": {
                    "delta": -0.60,
                    "gamma": 0.042,
                    "theta": -0.021,
                    "vega": 0.14,
                    "rho": -0.035,
                },
                "implied_volatility": 0.26,
                "last_quote": {
                    "ask": 6.10,
                    "bid": 5.80,
                    "bid_size": 12,
                    "ask_size": 10,
                },
                "open_interest": 3400,
            },
            {
                "option_id": "O:XOM230421C00105000",
                "break_even_price": 110.80,
                "day": {
                    "change": 2.10,
                    "change_percent": 1.95,
                    "close": 110.50,
                    "high": 111.50,
                    "low": 109.50,
                    "open": 110.00,
                    "volume": 1700,
                    "vwap": 110.40,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2023-04-21",
                    "shares_per_contract": 100,
                    "strike_price": 105.00,
                },
                "greeks": {
                    "delta": 0.81,
                    "gamma": 0.048,
                    "theta": -0.026,
                    "vega": 0.17,
                    "rho": 0.065,
                },
                "implied_volatility": 0.31,
                "last_quote": {
                    "ask": 6.80,
                    "bid": 6.50,
                    "bid_size": 14,
                    "ask_size": 12,
                },
                "open_interest": 3800,
            },
        ],
    },


        {
        "ticker": "JNJ",
        "options": [
            {
                "option_id": "O:JNJ240119C00165000",
                "break_even_price": 170.80,
                "day": {
                    "change": 1.90,
                    "change_percent": 1.15,
                    "close": 169.00,
                    "high": 171.00,
                    "low": 168.00,
                    "open": 168.50,
                    "volume": 2200,
                    "vwap": 169.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-01-19",
                    "shares_per_contract": 100,
                    "strike_price": 165.00,
                },
                "greeks": {
                    "delta": 0.68,
                    "gamma": 0.051,
                    "theta": -0.020,
                    "vega": 0.14,
                    "rho": 0.048,
                },
                "implied_volatility": 0.21,
                "last_quote": {
                    "ask": 6.00,
                    "bid": 5.80,
                    "bid_size": 25,
                    "ask_size": 22,
                },
                "open_interest": 4700,
            },
            {
                "option_id": "O:JNJ240616P00150000",
                "break_even_price": 147.00,
                "day": {
                    "change": -1.50,
                    "change_percent": -1.05,
                    "close": 148.50,
                    "high": 149.50,
                    "low": 147.50,
                    "open": 149.00,
                    "volume": 1600,
                    "vwap": 148.00,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2024-06-16",
                    "shares_per_contract": 100,
                    "strike_price": 150.00,
                },
                "greeks": {
                    "delta": -0.58,
                    "gamma": 0.045,
                    "theta": -0.022,
                    "vega": 0.13,
                    "rho": -0.046,
                },
                "implied_volatility": 0.20,
                "last_quote": {
                    "ask": 5.20,
                    "bid": 4.90,
                    "bid_size": 18,
                    "ask_size": 16,
                },
                "open_interest": 3300,
            },
            {
                "option_id": "O:JNJ241220C00180000",
                "break_even_price": 185.50,
                "day": {
                    "change": 2.50,
                    "change_percent": 1.40,
                    "close": 183.00,
                    "high": 186.00,
                    "low": 182.00,
                    "open": 183.50,
                    "volume": 1900,
                    "vwap": 183.80,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-12-20",
                    "shares_per_contract": 100,
                    "strike_price": 180.00,
                },
                "greeks": {
                    "delta": 0.71,
                    "gamma": 0.053,
                    "theta": -0.019,
                    "vega": 0.15,
                    "rho": 0.050,
                },
                "implied_volatility": 0.23,
                "last_quote": {
                    "ask": 8.20,
                    "bid": 7.90,
                    "bid_size": 20,
                    "ask_size": 17,
                },
                "open_interest": 5100,
            },
            {
                "option_id": "O:JNJ231117P00155000",
                "break_even_price": 153.00,
                "day": {
                    "change": -2.10,
                    "change_percent": -1.35,
                    "close": 154.50,
                    "high": 155.50,
                    "low": 153.50,
                    "open": 155.00,
                    "volume": 1300,
                    "vwap": 154.00,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2023-11-17",
                    "shares_per_contract": 100,
                    "strike_price": 155.00,
                },
                "greeks": {
                    "delta": -0.65,
                    "gamma": 0.047,
                    "theta": -0.021,
                    "vega": 0.14,
                    "rho": -0.042,
                },
                "implied_volatility": 0.22,
                "last_quote": {
                    "ask": 6.50,
                    "bid": 6.20,
                    "bid_size": 14,
                    "ask_size": 12,
                },
                "open_interest": 3600,
            },
            {
                "option_id": "O:JNJ230421C00175000",
                "break_even_price": 178.50,
                "day": {
                    "change": 1.80,
                    "change_percent": 1.05,
                    "close": 177.00,
                    "high": 178.50,
                    "low": 176.00,
                    "open": 176.50,
                    "volume": 1800,
                    "vwap": 177.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2023-04-21",
                    "shares_per_contract": 100,
                    "strike_price": 175.00,
                },
                "greeks": {
                    "delta": 0.77,
                    "gamma": 0.049,
                    "theta": -0.025,
                    "vega": 0.16,
                    "rho": 0.056,
                },
                "implied_volatility": 0.25,
                "last_quote": {
                    "ask": 6.80,
                    "bid": 6.50,
                    "bid_size": 15,
                    "ask_size": 13,
                },
                "open_interest": 3900,
            },
        ],
    },
        
    {
        "ticker": "HD",
        "options": [
            {
                "option_id": "O:HD240119C00300000",
                "break_even_price": 305.50,
                "day": {
                    "change": 2.10,
                    "change_percent": 0.70,
                    "close": 304.00,
                    "high": 306.50,
                    "low": 303.00,
                    "open": 303.50,
                    "volume": 2500,
                    "vwap": 304.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-01-19",
                    "shares_per_contract": 100,
                    "strike_price": 300.00,
                },
                "greeks": {
                    "delta": 0.65,
                    "gamma": 0.048,
                    "theta": -0.018,
                    "vega": 0.12,
                    "rho": 0.042,
                },
                "implied_volatility": 0.22,
                "last_quote": {
                    "ask": 6.50,
                    "bid": 6.20,
                    "bid_size": 20,
                    "ask_size": 18,
                },
                "open_interest": 4800,
            },
            {
                "option_id": "O:HD240616P00280000",
                "break_even_price": 277.00,
                "day": {
                    "change": -1.80,
                    "change_percent": -0.65,
                    "close": 278.50,
                    "high": 280.00,
                    "low": 277.50,
                    "open": 279.00,
                    "volume": 2200,
                    "vwap": 278.75,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2024-06-16",
                    "shares_per_contract": 100,
                    "strike_price": 280.00,
                },
                "greeks": {
                    "delta": -0.54,
                    "gamma": 0.043,
                    "theta": -0.020,
                    "vega": 0.13,
                    "rho": -0.039,
                },
                "implied_volatility": 0.20,
                "last_quote": {
                    "ask": 5.40,
                    "bid": 5.10,
                    "bid_size": 16,
                    "ask_size": 14,
                },
                "open_interest": 3700,
            },
            {
                "option_id": "O:HD241220C00325000",
                "break_even_price": 330.50,
                "day": {
                    "change": 3.20,
                    "change_percent": 0.95,
                    "close": 328.00,
                    "high": 331.00,
                    "low": 327.00,
                    "open": 327.50,
                    "volume": 2700,
                    "vwap": 328.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-12-20",
                    "shares_per_contract": 100,
                    "strike_price": 325.00,
                },
                "greeks": {
                    "delta": 0.72,
                    "gamma": 0.050,
                    "theta": -0.017,
                    "vega": 0.14,
                    "rho": 0.048,
                },
                "implied_volatility": 0.25,
                "last_quote": {
                    "ask": 8.90,
                    "bid": 8.50,
                    "bid_size": 22,
                    "ask_size": 20,
                },
                "open_interest": 5200,
            },
            {
                "option_id": "O:HD231117P00275000",
                "break_even_price": 272.00,
                "day": {
                    "change": -2.30,
                    "change_percent": -0.85,
                    "close": 273.50,
                    "high": 275.00,
                    "low": 272.50,
                    "open": 274.00,
                    "volume": 1800,
                    "vwap": 273.25,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2023-11-17",
                    "shares_per_contract": 100,
                    "strike_price": 275.00,
                },
                "greeks": {
                    "delta": -0.62,
                    "gamma": 0.046,
                    "theta": -0.019,
                    "vega": 0.11,
                    "rho": -0.037,
                },
                "implied_volatility": 0.19,
                "last_quote": {
                    "ask": 6.70,
                    "bid": 6.30,
                    "bid_size": 12,
                    "ask_size": 10,
                },
                "open_interest": 3400,
            },
            {
                "option_id": "O:HD230421C00310000",
                "break_even_price": 315.00,
                "day": {
                    "change": 2.70,
                    "change_percent": 0.85,
                    "close": 313.50,
                    "high": 316.00,
                    "low": 312.50,
                    "open": 313.00,
                    "volume": 2100,
                    "vwap": 313.75,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2023-04-21",
                    "shares_per_contract": 100,
                    "strike_price": 310.00,
                },
                "greeks": {
                    "delta": 0.70,
                    "gamma": 0.049,
                    "theta": -0.018,
                    "vega": 0.13,
                    "rho": 0.046,
                },
                "implied_volatility": 0.24,
                "last_quote": {
                    "ask": 7.10,
                    "bid": 6.80,
                    "bid_size": 18,
                    "ask_size": 16,
                },
                "open_interest": 4000,
            },
        ],
    },

    {
        "ticker": "JPM",
        "options": [
            {
                "option_id": "O:JPM240119C00140000",
                "break_even_price": 145.50,
                "day": {
                    "change": 1.30,
                    "change_percent": 0.90,
                    "close": 144.20,
                    "high": 145.00,
                    "low": 143.50,
                    "open": 143.80,
                    "volume": 2200,
                    "vwap": 144.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-01-19",
                    "shares_per_contract": 100,
                    "strike_price": 140.00,
                },
                "greeks": {
                    "delta": 0.64,
                    "gamma": 0.042,
                    "theta": -0.017,
                    "vega": 0.12,
                    "rho": 0.041,
                },
                "implied_volatility": 0.20,
                "last_quote": {
                    "ask": 5.80,
                    "bid": 5.50,
                    "bid_size": 18,
                    "ask_size": 20,
                },
                "open_interest": 4300,
            },
            {
                "option_id": "O:JPM240616P00150000",
                "break_even_price": 145.00,
                "day": {
                    "change": -2.10,
                    "change_percent": -1.35,
                    "close": 147.10,
                    "high": 148.50,
                    "low": 146.50,
                    "open": 147.80,
                    "volume": 1800,
                    "vwap": 147.20,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2024-06-16",
                    "shares_per_contract": 100,
                    "strike_price": 150.00,
                },
                "greeks": {
                    "delta": -0.52,
                    "gamma": 0.038,
                    "theta": -0.019,
                    "vega": 0.11,
                    "rho": -0.039,
                },
                "implied_volatility": 0.23,
                "last_quote": {
                    "ask": 6.20,
                    "bid": 5.90,
                    "bid_size": 14,
                    "ask_size": 16,
                },
                "open_interest": 3700,
            },
            {
                "option_id": "O:JPM241220C00160000",
                "break_even_price": 164.50,
                "day": {
                    "change": 3.20,
                    "change_percent": 2.05,
                    "close": 162.00,
                    "high": 163.50,
                    "low": 161.00,
                    "open": 161.20,
                    "volume": 2600,
                    "vwap": 162.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2024-12-20",
                    "shares_per_contract": 100,
                    "strike_price": 160.00,
                },
                "greeks": {
                    "delta": 0.70,
                    "gamma": 0.048,
                    "theta": -0.015,
                    "vega": 0.14,
                    "rho": 0.049,
                },
                "implied_volatility": 0.25,
                "last_quote": {
                    "ask": 7.50,
                    "bid": 7.10,
                    "bid_size": 16,
                    "ask_size": 14,
                },
                "open_interest": 5400,
            },
            {
                "option_id": "O:JPM231117P00145000",
                "break_even_price": 142.50,
                "day": {
                    "change": -1.80,
                    "change_percent": -1.20,
                    "close": 144.30,
                    "high": 145.20,
                    "low": 143.80,
                    "open": 144.00,
                    "volume": 2000,
                    "vwap": 144.10,
                },
                "details": {
                    "contract_type": "put",
                    "exercise_style": "American",
                    "expiration_date": "2023-11-17",
                    "shares_per_contract": 100,
                    "strike_price": 145.00,
                },
                "greeks": {
                    "delta": -0.58,
                    "gamma": 0.040,
                    "theta": -0.018,
                    "vega": 0.10,
                    "rho": -0.036,
                },
                "implied_volatility": 0.19,
                "last_quote": {
                    "ask": 5.70,
                    "bid": 5.40,
                    "bid_size": 10,
                    "ask_size": 12,
                },
                "open_interest": 3900,
            },
            {
                "option_id": "O:JPM230421C00155000",
                "break_even_price": 157.50,
                "day": {
                    "change": 2.10,
                    "change_percent": 1.35,
                    "close": 156.00,
                    "high": 157.50,
                    "low": 155.50,
                    "open": 155.80,
                    "volume": 2800,
                    "vwap": 156.50,
                },
                "details": {
                    "contract_type": "call",
                    "exercise_style": "American",
                    "expiration_date": "2023-04-21",
                    "shares_per_contract": 100,
                    "strike_price": 155.00,
                },
                "greeks": {
                    "delta": 0.69,
                    "gamma": 0.045,
                    "theta": -0.014,
                    "vega": 0.12,
                    "rho": 0.043,
                },
                "implied_volatility": 0.22,
                "last_quote": {
                    "ask": 6.40,
                    "bid": 6.10,
                    "bid_size": 18,
                    "ask_size": 16,
                },
                "open_interest": 4700,
            },
        ],
    }

]

In [6]:
from datetime import datetime, timedelta
import yfinance as yf

def adjust_expiration_dates(sample_data):
    for stock in sample_data:
        ticker = stock["ticker"]
        stock_obj = yf.Ticker(ticker)
        for option in stock["options"]:
            expiration_date = datetime.strptime(option["details"]["expiration_date"], "%Y-%m-%d").date()
            # Fetch valid trading dates
            history = stock_obj.history(
                start=(expiration_date - timedelta(days=10)).isoformat(),
                end=(expiration_date + timedelta(days=10)).isoformat()
            )
            trading_dates = history.index.date
            # Adjust to the closest prior trading date
            while expiration_date not in trading_dates:
                expiration_date -= timedelta(days=1)
            # Update expiration date in the dataset
            option["details"]["expiration_date"] = expiration_date.isoformat()

    return sample_data

# Preprocess the sample data
sample_data = adjust_expiration_dates(sample_data)

In [7]:
def test_get_similar_historical_options():
    """
    Test the get_similar_historical_options function using sample data with relaxed filters.
    """
    test_cases = [
        {"ticker": "AAPL", "expiry_days": 30, "strike_ratio_range": (0.5, 1.5), "volatility_range": (0.1, 0.35), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31)), "expected_min_results": 3},
        {"ticker": "XOM", "expiry_days": 60, "strike_ratio_range": (0.6, 1.4), "volatility_range": (0.15, 0.3), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31)), "expected_min_results": 3},
        {"ticker": "JNJ", "expiry_days": 120, "strike_ratio_range": (0.7, 1.3), "volatility_range": (0.1, 0.3), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31)), "expected_min_results": 3},
        {"ticker": "INVALID", "expiry_days": 30, "strike_ratio_range": (0.5, 1.5), "volatility_range": (0.1, 0.35), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31)), "expected_min_results": 0},
    ]

    for i, test in enumerate(test_cases):
        print(f"Running Test Case {i + 1}: {test['ticker']}")
        results = get_similar_historical_options(
            ticker=test["ticker"],
            expiry_days=test["expiry_days"],
            strike_ratio_range=test["strike_ratio_range"],
            volatility_range=test["volatility_range"],
            date_range=test["date_range"],
        )

        # Validate results
        if len(results) >= test["expected_min_results"]:
            print(f"Test Case {i + 1} Passed: Found {len(results)} matching options.")
        else:
            print(f"Test Case {i + 1} Failed: Expected at least {test['expected_min_results']} results but found {len(results)}.")

        # Debug output for the first few results
        if results:
            print("Sample Result:")
            print(results[0])
        print("-" * 50)

# Run the test cases
test_get_similar_historical_options()

Running Test Case 1: AAPL
Using sample data...
Total matching options from sample data: 5
Test Case 1 Passed: Found 5 matching options.
Sample Result:
{'option_id': 'O:AAPL240119C00150000', 'break_even_price': 155.5, 'day': {'change': 2.3, 'change_percent': 1.5, 'close': 153.2, 'high': 155.6, 'low': 151.5, 'open': 152.0, 'volume': 1400, 'vwap': 153.0}, 'details': {'contract_type': 'call', 'exercise_style': 'American', 'expiration_date': '2024-01-19', 'shares_per_contract': 100, 'strike_price': 150.0}, 'greeks': {'delta': 0.68, 'gamma': 0.045, 'theta': -0.02, 'vega': 0.14, 'rho': 0.04}, 'implied_volatility': 0.24, 'last_quote': {'ask': 5.1, 'bid': 4.9, 'bid_size': 20, 'ask_size': 15}, 'open_interest': 3400, 'ticker': 'AAPL'}
--------------------------------------------------
Running Test Case 2: XOM
Using sample data...
Total matching options from sample data: 4
Test Case 2 Passed: Found 4 matching options.
Sample Result:
{'option_id': 'O:XOM240119C00100000', 'break_even_price': 105.5, 

# run_backtest.ipynb

In [8]:
import yfinance as yf
from datetime import datetime, timedelta, date
from typing import List, Dict, Optional

In [9]:


def run_backtest(similar_options: List[Dict], strategy_params: Optional[Dict] = None) -> Dict:
    """
    Run a backtest on historically similar options, evaluating based on expiration prices.
    """
    # Initialize variables
    total_trades = 0
    winning_trades = 0
    losing_trades = 0
    cumulative_pnl = 0.0
    trade_log = []
    funds = [100_000]  # Starting capital
    transaction_cost = strategy_params.get("transaction_cost", 1.0) if strategy_params else 1.0
    max_drawdown = 0.0
    peak_value = 0.0
    max_funds = 0.0
    min_funds = float("inf")

    # Iterate over each option in the similar options list
    for trade_id, opt in enumerate(similar_options, 1):
        try:
            # Extract necessary details from the option
            option_type = opt["details"]["contract_type"]
            strike_price = opt["details"]["strike_price"]
            shares_per_contract = opt["details"]["shares_per_contract"]
            expiration_date = datetime.strptime(opt["details"]["expiration_date"], "%Y-%m-%d").date()
            ticker = opt["option_id"].split(":")[1]

            # Fetch closest trading day price at expiration using yfinance
            def get_closest_trading_date(stock_ticker: str, date_to_check: date) -> date:
                stock = yf.Ticker(stock_ticker)
                historical_data = stock.history(
                    start=(date_to_check - timedelta(days=10)).isoformat(),
                    end=(date_to_check + timedelta(days=10)).isoformat()
                )
                trading_dates = historical_data.index.date

                while date_to_check not in trading_dates:
                    date_to_check -= timedelta(days=1)

                return date_to_check

            # Adjust expiration date to the closest trading date
            adjusted_expiration_date = get_closest_trading_date(opt.get("ticker"), expiration_date)

            stock = yf.Ticker(opt.get("ticker"))
            history = stock.history(
                start=adjusted_expiration_date.isoformat(),
                end=(adjusted_expiration_date + timedelta(days=1)).isoformat()
            )

            if history.empty:
                print(f"Unable to fetch price for {ticker} on adjusted expiration date {adjusted_expiration_date}")
                continue

            underlying_price_at_expiry = history['Close'].iloc[0]

            # Calculate profit/loss based on option type
            if option_type == "call":
                pnl = max(0, (underlying_price_at_expiry - strike_price)) * shares_per_contract
            elif option_type == "put":
                pnl = max(0, (strike_price - underlying_price_at_expiry)) * shares_per_contract
            else:
                pnl = 0  # Unknown contract type

            # Subtract transaction cost
            pnl -= transaction_cost
            cumulative_pnl += pnl
            funds.append(funds[-1] + pnl)

            # Update win/loss metrics
            total_trades += 1
            if pnl > 0:
                winning_trades += 1
            else:
                losing_trades += 1

            # Update drawdown metrics
            peak_value = max(peak_value, funds[-1])
            drawdown = (peak_value - funds[-1]) / peak_value if peak_value > 0 else 0
            max_drawdown = max(max_drawdown, drawdown)

            # Log trade details
            trade_log.append({
                "trade_id": trade_id,
                "ticker": ticker,
                "option_type": option_type,
                "strike_price": strike_price,
                "underlying_price_at_expiry": underlying_price_at_expiry,
                "expiration_date": expiration_date,
                "pnl": pnl,
                "transaction_cost": transaction_cost,
            })
        except Exception as e:
            print(f"Error processing option {trade_id}: {e}")

    # Calculate overall metrics
    success_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
    ending_capital = funds[-1]
    max_funds = max(funds)
    min_funds = min(funds)
    avg_pnl = cumulative_pnl / total_trades if total_trades > 0 else 0
    recommendation = "BUY" if success_rate > 60 and avg_pnl > 0 else "AVOID"

    return {
        "recommendation": recommendation,
        "success_rate": success_rate,
        "cumulative_pnl": cumulative_pnl,
        "total_trades": total_trades,
        "winning_trades": winning_trades,
        "losing_trades": losing_trades,
        "avg_pnl": avg_pnl,
        "max_drawdown": max_drawdown * 100,  # Convert to percentage
        "starting_capital": funds[0],
        "ending_capital": ending_capital,
        "max_funds": max_funds,
        "min_funds": min_funds,
        "trade_log": trade_log,
        "funds_by_date": funds,
    }

In [10]:
def test_run_backtest():
    """
    Test the run_backtest function by first fetching historical options using
    get_similar_historical_options and then evaluating the results using run_backtest.
    """

    # Define test cases for the stocks
    test_cases = [
        {"ticker": "AAPL", "expiry_days": 30, "strike_ratio_range": (0.5, 1.5), "volatility_range": (0.1, 0.35), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31))},
        {"ticker": "XOM", "expiry_days": 60, "strike_ratio_range": (0.6, 1.4), "volatility_range": (0.15, 0.3), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31))},
        {"ticker": "JNJ", "expiry_days": 120, "strike_ratio_range": (0.7, 1.3), "volatility_range": (0.1, 0.3), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31))},
        {"ticker": "HD", "expiry_days": 90, "strike_ratio_range": (0.6, 1.4), "volatility_range": (0.15, 0.3), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31))},
        {"ticker": "JPM", "expiry_days": 60, "strike_ratio_range": (0.5, 1.5), "volatility_range": (0.1, 0.35), 
         "date_range": (date(2023, 1, 1), date(2024, 12, 31))},
    ]

    # Define strategy parameters for backtesting
    strategy_params = {
        "transaction_cost": 1.0  # Example fixed transaction cost
    }

    # Run each test case
    for i, test in enumerate(test_cases, 1):
        print(f"Running Backtest Test Case {i}: {test['ticker']}")

        # Fetch similar historical options for the given stock
        similar_options = get_similar_historical_options(
            ticker=test["ticker"],
            expiry_days=test["expiry_days"],
            strike_ratio_range=test["strike_ratio_range"],
            volatility_range=test["volatility_range"],
            date_range=test["date_range"],
        )

        # If no options were found, skip the backtest
        if not similar_options:
            print(f"No similar historical options found for {test['ticker']}. Skipping backtest.")
            print("-" * 50)
            continue

        # Run backtest on the fetched options
        backtest_results = run_backtest(similar_options, strategy_params)

        # Print the backtest results
        print(f"Backtest Results for {test['ticker']}:")
        print(f"Recommendation: {backtest_results['recommendation']}")
        print(f"Success Rate: {backtest_results['success_rate']}%")
        print(f"Cumulative P/L: {backtest_results['cumulative_pnl']}")
        print(f"Total Trades: {backtest_results['total_trades']}")
        print(f"Winning Trades: {backtest_results['winning_trades']}")
        print(f"Losing Trades: {backtest_results['losing_trades']}")
        print(f"Max Drawdown: {backtest_results['max_drawdown']}%")
        print(f"Starting Capital: {backtest_results['starting_capital']}")
        print(f"Ending Capital: {backtest_results['ending_capital']}")
        print(f"Max Funds: {backtest_results['max_funds']}")
        print(f"Min Funds: {backtest_results['min_funds']}")
        print(f"Average P/L per Trade: {backtest_results['avg_pnl']}")
        print("-" * 50)

# Run the test
test_run_backtest()

Running Backtest Test Case 1: AAPL
Using sample data...
Total matching options from sample data: 5
Backtest Results for AAPL:
Recommendation: AVOID
Success Rate: 40.0%
Cumulative P/L: 13506.611938476562
Total Trades: 5
Winning Trades: 2
Losing Trades: 3
Max Drawdown: 0.001761980845192637%
Starting Capital: 100000
Ending Capital: 113506.61193847656
Max Funds: 113508.61193847656
Min Funds: 100000
Average P/L per Trade: 2701.3223876953125
--------------------------------------------------
Running Backtest Test Case 2: XOM
Using sample data...
Total matching options from sample data: 4
Backtest Results for XOM:
Recommendation: AVOID
Success Rate: 25.0%
Cumulative P/L: 846.2235412597656
Total Trades: 4
Winning Trades: 1
Losing Trades: 3
Max Drawdown: 0.002000020000200002%
Starting Capital: 100000
Ending Capital: 100846.22354125977
Max Funds: 100846.22354125977
Min Funds: 99997.0
Average P/L per Trade: 211.5558853149414
--------------------------------------------------
Running Backtest Test

# Monte_carlo_option_pricing_model.ipynb